<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_paper1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependencies

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
import math

# Import dataset

In [3]:
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")
dataset_lab = pd.read_csv("sample_data.csv")

In [4]:
dataset = pd.concat([dataset_lab, dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)

# Data processing

In [5]:
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

In [6]:
low_pass_filter = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter = np.zeros((X.shape[0], X.shape[1]))

for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter[i][j] = X[i][j]
            high_pass_filter[i][j] = X[i][j]
        else:
            low_pass_filter[i][j] = 0.8*X[i-1][j] + (1-0.8)*X[i][j]
            high_pass_filter[i][j] = X[i][j] - low_pass_filter[i][j]

## Frame preparation

In [7]:
Fs = 4
frame_size = Fs*20 # 80 # 20 seconds of data
hop_size =  Fs*10

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame, low_pass_label = get_frames(low_pass_filter, frame_size, hop_size, Y)

## Feature calculation

In [8]:
mean = np.mean(low_pass_frame, axis = 1)  # mean along (x,y,z) axis
std = np.std(low_pass_frame, axis = 1)  # standard deviation along (x,y,z) axis

In [9]:
rms_x = []  # root mean square along x axis
value = 0
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][0] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][1] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][2] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0


In [10]:
lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = row_frame.shape[0], row_frame.shape[1], row_frame.shape[2]
acc_vector = np.reshape(row_frame,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

In [11]:
avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])
    
    value = value / (frame_size / polling_rate)
    avc.append(value)

In [13]:
max_min = []

for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))

In [14]:
angle = np.zeros((low_pass_frame.shape[0],low_pass_frame.shape[1],low_pass_frame.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = low_pass_frame[i,j,0]**2 + low_pass_frame[i,j,1]**2 + low_pass_frame[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(low_pass_frame[i,j,0]/length)
            angle_y = math.acos(low_pass_frame[i,j,1]/length)
            angle_z = math.acos(low_pass_frame[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))

In [15]:
input_vector = np.zeros((row_frame.shape[0], angle.shape[1] + 11))  # input vector for the algorithm
for i in range(input_vector.shape[0]):
    input_vector[i][0] = mean[i][0]
    input_vector[i][1] = mean[i][1]
    input_vector[i][2] = mean[i][2] 
    input_vector[i][3] = std[i][0]
    input_vector[i][4] = std[i][1]
    input_vector[i][5] = std[i][2]
    input_vector[i][6] = rms_x[i]
    input_vector[i][7] = rms_y[i]
    input_vector[i][8] = rms_z[i]
    input_vector[i][9] = max_min[i]
    input_vector[i][10] = avc[i]

for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        input_vector[i][j+11] = angle[i][j]


In [16]:
label = row_label

# Feature scaling

In [17]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)

# Train test split

In [18]:
X_train, X_val, y_train, y_val = train_test_split(input_vector, label, test_size = 0.2, random_state = 0, stratify = label)

## Checking ambiguity

In [23]:
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[0]):
    if (i != j) and np.array_equal(X_train[i],X_train[j]):
      print("i = ",i, "label = ", y_train[i], "j = ",j,"label = ",y_train[j])


i =  85 label =  11.0 j =  152 label =  9.0
i =  85 label =  11.0 j =  402 label =  7.0
i =  85 label =  11.0 j =  487 label =  7.0
i =  85 label =  11.0 j =  814 label =  1.0
i =  85 label =  11.0 j =  1183 label =  9.0
i =  85 label =  11.0 j =  1763 label =  3.0
i =  85 label =  11.0 j =  2024 label =  8.0
i =  85 label =  11.0 j =  2328 label =  9.0
i =  85 label =  11.0 j =  2517 label =  5.0
i =  85 label =  11.0 j =  2568 label =  11.0
i =  85 label =  11.0 j =  2622 label =  7.0
i =  85 label =  11.0 j =  2871 label =  3.0
i =  85 label =  11.0 j =  2915 label =  10.0
i =  85 label =  11.0 j =  2966 label =  9.0
i =  85 label =  11.0 j =  3027 label =  6.0
i =  85 label =  11.0 j =  3060 label =  11.0
i =  85 label =  11.0 j =  3074 label =  6.0
i =  85 label =  11.0 j =  3115 label =  3.0
i =  85 label =  11.0 j =  3284 label =  1.0
i =  85 label =  11.0 j =  3681 label =  11.0
i =  85 label =  11.0 j =  3748 label =  3.0
i =  85 label =  11.0 j =  3907 label =  1.0
i =  85 la

KeyboardInterrupt: ignored

In [ ]:
%debug

# Algorithm

## Random forest

In [18]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [19]:
y_pred = classifier.predict(X_val)

In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, y_val)

In [24]:
cm

array([[  467,    30,    44,    54,    59,    73,    35,    16,    23,
           12,    20,    22],
       [   36,   103,    53,    65,    30,    28,    39,    23,    29,
           19,    26,    32],
       [   95,   106,  1297,   306,   470,    80,   164,    83,    55,
           40,    63,   125],
       [  526,   298,  1053,  9716,  8677,   918,  1762,   292,   279,
          263,   294,   694],
       [  843,   336,  3205, 10510, 45129,   299,  2421,   260,   362,
          351,   340,  1390],
       [   78,    34,    41,   593,    32,   313,    67,    43,    33,
           22,    39,    33],
       [   33,    38,    56,   175,   155,    45,  1794,    36,    50,
           37,    32,    63],
       [   16,    18,    26,    26,    36,    21,    32,   186,    37,
           21,    30,    29],
       [    9,    15,    22,    17,    27,    20,    18,    41,   131,
           43,    21,    12],
       [   11,    16,    14,    22,    40,    14,    46,    47,    59,
          454,    23

In [22]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [23]:
accuracy

0.5974700126108335

## Neural Network Algorithm

## Oversampling to balance the data

Different kinds of oversampling processes are performed below. Choose any one of them

### Random oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_train, y_train = ros.fit_resample(X_train, y_train)

### Adaptive synthetic sampling methond

In [ ]:
 from imblearn.over_sampling ADASYN
  X_train, y_train = ADASYN().fit_resample(X_train, y_train)

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
#model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=324, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)

## CNN

In [ ]:
X_train = X_train.reshape(-1, 251, 1, 1)
X_val = X_val.reshape(-1, 251, 1, 1)


In [ ]:
model = Sequential()

model.add(Conv2D(16, (2, 2), activation = 'relu', padding = 'same', input_shape = X_train[0].shape))
model.add(Dropout(0.1))
model.add(Conv2D(32, (2, 2), activation='relu', padding= 'same'))
model.add(Dropout(0.2))
#model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)